In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Thymoma/GSE42977'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Sequential Binary Gene-Ratio Tests Define a Novel Molecular Diagnostic Strategy for Malignant Pleural Mesothelioma"
!Series_summary	"The gene-expression ratio  technique was used to design a molecular signature to diagnose MPM from among other potentially confounding diagnoses and differentiate the epithelioid from the sarcomatoid histological subtype of MPM."
!Series_overall_design	"Microarray analysis was performed on 113 specimens including MPMs and a spectrum of tumors and benign tissues comprising the differential diagnosis of MPM.  A sequential combination of binary gene-expression ratio tests was developed to discriminate MPM from other thoracic malignancies .  This method was compared to other bioinformatic tools and this signature was validated  in an independent set of 170 samples.  Functional enrichment analysis was performed to identify differentially expressed probes."
Sample Characteristics Dictionary:
{0: ['tissue: control', 'tissue

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # Based on series summary indicating microarray analysis (gene-expression)

# Determine the availability of variables in the dataset
sample_characteristics = {
    0: ['tissue: control', 'tissue: Spindle Cell Sarcoma', 'tissue: Sarcoma', 'tissue: Metastatic Melanoma', 
        'tissue: Pleomorphic Sarcoma', 'tissue: Renal Cell Carcinoma-Clear Cell', 'tissue: Synovial Sarcoma', 
        'tissue: Metastatic Thymoma', 'tissue: Metastatic Prostate Cancer', 'tissue: Stomach Cancer-Stromal Sarcoma', 
        'tissue: Non-Hodgkins Lymphoma', 'tissue: Hemangioendothelioma', 'tissue: Papillary Thyroid Carcinoma', 
        'tissue: Metastatic Thyroid Cancer', 'tissue: Lymphocytic Lymphoma', 'tissue: Thymoma', 'tissue: Melanoma-Malignant', 
        'tissue: Hemangiopericytoma', 'tissue: Thyroid Carcinoma', 'tissue: Monophasic Synovial Sarcoma', 
        'tissue: Metastatic Alveolar Soft Part Sarcoma', 'tissue: Metastatic Meningeal Hemangiopericytoma', 
        'tissue: Follicular Lymphoma', 'tissue: Rhabdomyosarcoma', 'tissue: Myofibrosarcoma', 'tissue: Renal Cell Carcinoma - Chromophobe', 
        'tissue: MPM Epithelioid', 'tissue: MPM Sarcomatoid', 'tissue: MPM Biphasic', 'tissue: Normal Lung']
}

# Check for 'Thymoma'
if 'tissue: Thymoma' in sample_characteristics[0]:
    trait_row = 0

# Check for 'age' (none found in characteristics, so setting to None)
age_row = None

# Check for 'gender' (none found in characteristics, so setting to None)
gender_row = None

# Define conversion functions
def convert_trait(value):
    value = value.split(":")[-1].strip()
    return 1 if value == "Thymoma" else 0

# No conversion needed for age_row and gender_row as they are not available
convert_age = None
convert_gender = None

# Save cohort information
save_cohort_info('GSE42977', './preprocessed/Thymoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Thymoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Thymoma/trait_data/GSE42977.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1054230': [0], 'GSM1054231': [0], 'GSM1054232': [0], 'GSM1054233': [0], 'GSM1054234': [0], 'GSM1054235': [0], 'GSM1054236': [0], 'GSM1054237': [0], 'GSM1054238': [0], 'GSM1054239': [0], 'GSM1054240': [0], 'GSM1054241': [0], 'GSM1054242': [0], 'GSM1054243': [0], 'GSM1054244': [0], 'GSM1054245': [0], 'GSM1054246': [0], 'GSM1054247': [0], 'GSM1054248': [0], 'GSM1054249': [0], 'GSM1054250': [0], 'GSM1054251': [0], 'GSM1054252': [0], 'GSM1054253': [0], 'GSM1054254': [0], 'GSM1054255': [0], 'GSM1054256': [1], 'GSM1054257': [0], 'GSM1054258': [0], 'GSM1054259': [0], 'GSM1054260': [0], 'GSM1054261': [0], 'GSM1054262': [0], 'GSM1054263': [0], 'GSM1054264': [0], 'GSM1054265': [1], 'GSM1054266': [0], 'GSM1054267': [0], 'GSM1054268': [0], 'GSM1054269': [0], 'GSM1054270': [0], 'GSM1054271': [1], 'GSM1054272': [0], 'GSM1054273': [0], 'GSM1054274': [0], 'GSM1054275': [0], 'GSM1054276': [0], 'GSM1054277': [1], 'GSM1054278': [0], 'GSM1054279': [0], 'GSM1054280': [0], 'GSM1054281': [0], 'GSM1054282

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_10000', 'ILMN_100000', 'ILMN_100007', 'ILMN_100009', 'ILMN_10001',
       'ILMN_100010', 'ILMN_10002', 'ILMN_100028', 'ILMN_100030',
       'ILMN_100031', 'ILMN_100034', 'ILMN_100037', 'ILMN_10004', 'ILMN_10005',
       'ILMN_100054', 'ILMN_100059', 'ILMN_10006', 'ILMN_100075',
       'ILMN_100079', 'ILMN_100083'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'GB_ACC': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'SEQUENCE': ['CTCTCTAAAGGGACAACAGAGTGGACAGTCAAGGAACTCCACATATTCAT', 'GGGGTCAAGCCCAGGTGAAATGTGGATTGGAAAAGTGCTTCCCTTGCCCC', 'CCAGGCTGTAAAAGCAAAACCTCGTATCAGCTCTGGAACAATACCTGCAG', 'CCAGACAGGAAGCATCAAGCCCTTCAGGAAAGAATATGCGAGAGTGCTGC', 'TGTGCAGAAAGCTGATGGAAGGGAGAAAGAATGGAAGTGGGTCACACAGC'], 'Definition': ['UI-CF-EC0-abi-c-12-0-UI.s1 UI-CF-EC0 Homo sapiens cDNA clone UI-CF-EC0-abi-c-12-0-UI 3, mRNA sequence', 'PREDICTED: Homo sapiens similar to spectrin domain with coiled-coils 1 (LOC441782), mRNA.', 'Homo sapiens jumonji domain containing 1A (JMJD1A), mRNA.', 'hi56g05.x1 Soares_NFL_T_GBC_S1 Homo sapiens cDNA clone IMAGE:2976344 3, mRNA sequence', 'wk77d04.x1 NCI_CGAP_Pan1 Homo sapiens cDNA clone IMAGE:2421415 3, mRNA sequence'], 'Ontology': [nan, nan, nan, nan, na

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify keys for identifier and gene symbol
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the actual gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Thymoma/gene_data/GSE42977.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Thymoma')

# 4. Save the cohort information.
save_cohort_info('GSE42977', './preprocessed/Thymoma/cohort_info.json', True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Thymoma/GSE42977.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Thymoma', the least common label is '1.0' with 5 occurrences. This represents 4.27% of the dataset.
The distribution of the feature 'Thymoma' in this dataset is fine.

